Feature selection

BERT integration

In [1]:
#pip install torch


In [2]:
#pip install transformers[torch]

In [3]:
#pip install scikit-learn

In [4]:
#pip install pandas

In [5]:
#pip install datasets

In [6]:
#pip install AutoModel

In [7]:
#pip install tensorflow

In [8]:
#pip install tf-keras

distilbert-base-uncased because:
- pre-trained BERT model
- text classification
- distilled BERT model for smaller dataset
- slightly lesser performance than bert-base-uncased
- switch to bert-base-uncased for higher accuracy and if we have more computational resources; it will be slower

maybe not BERT? Any BERT models max text length is 512 and the news texts are longer


In [9]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import torch
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
# from datasets import Dataset
from transformers import AutoTokenizer # Hugging Face Transformers
from transformers import AutoModel
#from transformers import TFAutoModelForSequenceClassification
#from tensorflow.keras.optimizers import Adam
#from sklearn.metrics import accuracy_score


In [10]:
news_df = pd.read_csv('fake_or_real_news.csv') # TODO: Get cleaned data instead
news_df = news_df.drop(columns=['text'])
label_encoder = preprocessing.LabelEncoder()
news_df['label'] = label_encoder.fit_transform(news_df['label'].tolist())

In [11]:
train_df, test_df = train_test_split(news_df, test_size=0.2)

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel, AdamW

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModel.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english") # Example news data (replace with your actual dataset)
inputs = tokenizer(news_df['title'].to_list(), return_tensors='pt', padding=True, truncation=True, max_length=512)
optimizer = AdamW(model.parameters(), lr=5e-5)

epoch_accuracy = []

for epoch in range(3):



    with torch.no_grad():
        outputs = model(**inputs)
        cls_embeddings = outputs.last_hidden_state[:, 0, :]


    avg_embedding_magnitude = torch.norm(cls_embeddings, dim=1).mean().item()
    epoch_accuracy.append(avg_embedding_magnitude)

    print(f"Epoch {epoch + 1}: Average CLS Embedding Magnitude = {avg_embedding_magnitude}")


plt.figure(figsize=(8, 6))
plt.plot(range(1, len(epoch_accuracy) + 1), epoch_accuracy, marker='o', linestyle='-', color='b')
plt.title("CLS Embedding Change Over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Average CLS Embedding Magnitude")
plt.grid(True)
plt.show()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel, AdamW
# Load the pre-trained tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModel.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english") # Example news data (replace with your actual dataset)
inputs = tokenizer(news_df['title'].to_list(), return_tensors='pt', padding=True, truncation=True, max_length=512)
optimizer = AdamW(model.parameters(), lr=5e-5)
model.train()  # Set the model to training mode

#here is where I am going to make the loop for the chart of epochs

epoch_accuracy = []
for epoch in range(3):
    optimizer.zero_grad()  # Zero out previous gradients
    # Forward pass: Get model outputs
    with torch.no_grad(): # Since we're just extracting embeddings, no need to compute gradients
        outputs = model(**inputs) # Extract the CLS token embeddings (usually the first token)
        cls_embeddings = outputs.last_hidden_state[:, 0, :].numpy()

Using logistic regression for classification

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(cls_embeddings, news_df['label'].to_list(), test_size=0.2, random_state=42)

clf = LogisticRegression(max_iter=1000) # TODO: try knn
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Using KNN-3 for classification

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    cls_embeddings,
    news_df['label'].to_list(),
    test_size=0.2,
    random_state=42
)


clf = KNeighborsClassifier(n_neighbors=3)


clf.fit(X_train, y_train)


y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)